In [16]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=ConvergenceWarning)

import pandas
import csv
import matplotlib.pyplot as plt

#preprocesamiento
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std
import numpy as np
seed = 7
np.random.seed(seed)

# modelos
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import StackingClassifier

#guardar modelos
from sklearn.externals import joblib

#evaluacion
from sklearn.metrics import *
path = "data/"

In [48]:
data = pandas.read_csv("menortodos30.csv", decimal=".")


### Una vez preprocesado el dataset se debe ejecutar Training_Test_Data.py para crear 10 datasets para entrenamientos y test de cada modelo.

# Carga de datos

In [17]:
train_folds  = []
test_folds = []
for i in range(10):
    train = pandas.read_csv(path + "train_"+ str(i) + ".csv", decimal=".")
    col_mask=train.isnull().any(axis=0) 
    row_mask=train.isnull().any(axis=1)
    train_folds.append(train.loc[-row_mask,-col_mask])
    test = pandas.read_csv(path + "test_"+ str(i) + ".csv", decimal=".")
    col_mask=test.isnull().any(axis=0) 
    row_mask=test.isnull().any(axis=1)
    test_folds.append(test.loc[-row_mask,-col_mask])

PCA

In [45]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()# Fit on training set only.
train_x = scaler.fit_transform(train_folds[0].drop(columns=["hospital_death"]))
pca = PCA()
train_XX = pca.fit_transform(train_x)

In [41]:
pca.explained_variance_ratio_

array([8.90838550e-02, 4.73771688e-02, 3.96853284e-02, 3.32148441e-02,
       3.10227622e-02, 2.68910478e-02, 2.47573191e-02, 2.28419213e-02,
       2.18880573e-02, 1.99563467e-02, 1.85378599e-02, 1.74435355e-02,
       1.65703953e-02, 1.58080544e-02, 1.56515189e-02, 1.51927043e-02,
       1.40027084e-02, 1.36657737e-02, 1.31459245e-02, 1.30020251e-02,
       1.25345845e-02, 1.19531540e-02, 1.18575710e-02, 1.14886120e-02,
       1.12821670e-02, 1.10625199e-02, 1.06799220e-02, 1.06032356e-02,
       1.02839806e-02, 1.01002110e-02, 9.79993141e-03, 9.03591413e-03,
       8.94872637e-03, 8.61131250e-03, 8.50897038e-03, 8.35645304e-03,
       8.11049408e-03, 7.92903363e-03, 7.86557559e-03, 7.57126201e-03,
       7.40541845e-03, 7.26181691e-03, 7.03970155e-03, 6.83566786e-03,
       6.73505022e-03, 6.66593136e-03, 6.46626486e-03, 6.39283600e-03,
       6.34217704e-03, 6.28247713e-03, 6.19592418e-03, 6.16232659e-03,
       6.13331877e-03, 6.09939431e-03, 6.09658288e-03, 6.07837721e-03,
      

In [46]:
pca = PCA(n_components=5)
train_x = pca.fit_transform(train_folds[0].drop(columns=["hospital_death"]))
test_x = pca.fit_transform(test_folds[0].drop(columns=["hospital_death"]))
train_x = pandas.DataFrame(data=train_x[0:,0:], index=[i for i in range(train_x.shape[0])])
test_x = pandas.DataFrame(data=test_x[0:,0:], index=[i for i in range(test_x.shape[0])])
train_x

,0,1,2,3,4
0,-133.425467,416.729841,214.172404,76.781853,190.304880
1,-55.128765,378.932372,196.200346,125.988619,-127.472633
2,149.936581,416.423310,102.112823,-67.751090,-44.491560
3,46.582450,417.997994,-35.820739,144.959881,-67.643310
4,853.368060,412.614716,21.949534,-129.034877,8.641655
5,-330.798884,424.140374,41.119212,66.588264,45.032284
6,-133.769249,418.740881,101.852264,383.259601,56.119050
7,-52.105986,417.452705,-45.212036,11.724102,-50.618332
8,163.912747,425.815492,92.869646,142.535378,12.854047
9,-52.735063,396.333527,-34.471108,102.953351,-136.699888


Random forest

In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

classifier = RandomForestClassifier(max_depth=4, random_state=0)
classifier.fit(train_x, train_folds[0]["hospital_death"])

# Predicting the Test set results
y_pred = classifier.predict(test_x)
result = classification_report(test_folds[0]["hospital_death"], y_pred, output_dict=True)
recall = result["1"]["recall"]
recalln = result["0"]["recall"]
print("Recall SGB " + str(i) + ": Positive-> " + str(recall) +"\t Negative-> " + str(recalln))


Recall SGB 9: Positive-> 0.0	 Negative-> 1.0


/media/mdleiton/Respaldo/repositorios/env/lib/python3.5/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [73]:
data3 =data2.drop(columns=["hospital_death"])
data3

,leukemia,lymphoma,ethnicity_encoded,ethnicity_0,ethnicity_1,ethnicity_2,ethnicity_3,ethnicity_4,ethnicity_5,gender_encoded,...,h1_spo2_min,h1_spo2_max,h1_resprate_max,h1_mbp_noninvasive_min,h1_sysbp_max,h1_sysbp_noninvasive_min,h1_sysbp_min,h1_temp_min,d1_bun_max,h1_temp_max
0,0,0,2,0.0,0.0,1.0,0.0,0.0,0.0,1,...,74.0,100.0,26.0,85.0,131.0,115.0,115.0,37.5,31.0,39.5
1,0,0,2,0.0,0.0,1.0,0.0,0.0,0.0,0,...,70.0,95.0,31.0,57.0,95.0,71.0,71.0,36.3,11.0,36.3
2,0,0,2,0.0,0.0,1.0,0.0,0.0,0.0,0,...,91.0,98.0,20.0,83.0,148.0,124.0,124.0,36.7,19.0,36.7
3,0,0,2,0.0,0.0,1.0,0.0,0.0,0.0,0,...,99.0,100.0,12.0,79.0,136.0,115.0,106.0,34.8,19.0,35.6
4,0,0,2,0.0,0.0,1.0,0.0,0.0,0.0,1,...,100.0,100.0,21.0,92.0,130.0,120.0,120.0,36.6,19.0,36.7
5,0,0,2,0.0,0.0,1.0,0.0,0.0,0.0,1,...,97.0,97.0,12.0,111.0,143.0,143.0,143.0,36.7,13.0,36.7
6,0,0,2,0.0,0.0,1.0,0.0,0.0,0.0,0,...,100.0,100.0,18.0,117.0,191.0,115.0,163.0,35.0,18.0,36.8
7,0,0,2,0.0,0.0,1.0,0.0,0.0,0.0,1,...,92.0,96.0,28.0,60.0,119.0,106.0,106.0,38.5,48.0,38.5
8,0,0,2,0.0,0.0,1.0,0.0,0.0,0.0,1,...,97.0,97.0,24.0,71.0,104.0,98.0,98.0,36.9,15.0,36.9
9,0,0,2,0.0,0.0,1.0,0.0,0.0,0.0,1,...,96.0,100.0,29.0,77.0,135.0,103.0,103.0,36.9,10.0,36.9


In [3]:
train_folds[0]

,age,creatinine_apache,hematocrit_apache,map_apache,sodium_apache,temp_apache,wbc_apache,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,hospital_death
0,62.000000,0.70,27.200000,146.000000,136.0,38.90,18.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,63.000000,0.98,32.988739,43.000000,138.0,36.90,10.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,35.000000,0.40,36.700000,122.000000,132.0,37.00,17.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,22.000000,0.98,32.988739,107.000000,138.0,36.33,10.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,80.000000,1.70,29.900000,200.000000,153.0,36.40,14.20,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
5,59.000000,0.30,34.900000,63.000000,137.0,36.39,9.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,71.000000,0.98,32.988739,114.000000,138.0,36.60,10.40,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
7,50.000000,1.19,34.700000,120.000000,135.0,39.10,9.89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8,34.000000,2.10,37.400000,53.000000,128.0,36.20,7.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9,62.309516,0.98,32.988739,54.000000,138.0,36.50,10.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


# Cost-sensitive  Naive Classifier

In [16]:
# fuente  -> https://machinelearningmastery.com/naive-classifiers-imbalanced-classification-metrics/
model_NC = []

for i in range(len(train_folds)):
    model =  GaussianNB()
    model.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"])
    joblib.dump(model, 'NC'+ str(i) +'.pkl') 
    model_NC.append(model)
    predict = model.predict(test_folds[i].drop(columns=["hospital_death"]))

# Cost-sensitive  RANDOM FOREST

In [17]:
model_RF = []

for i in range(len(train_folds)):
    model = RandomForestClassifier(max_depth=3, random_state=0, class_weight='balanced', n_estimators=50)
    d = (train_folds[i]).drop(columns=["hospital_death"])
    model.fit(d, train_folds[i]["hospital_death"])
    joblib.dump(model, 'RF'+ str(i) +'.pkl') 
    model_RF.append(model)

# Cost-sensitive  SUPPORT VECTOR MACHINE

In [4]:
#fuente ->  https://machinelearningmastery.com/cost-sensitive-svm-for-imbalanced-classification/
model_SVM = []

for i in range(9, len(train_folds)):
    model = SVC(gamma='scale', class_weight='balanced')
    model.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"])
    joblib.dump(model, 'SVM'+ str(i) +'.pkl') 
    model_SVM.append(model)

Recall 9 fold: 0.765963
mean Recall : 0.7659625253610216


# Cost-sensitive  LOGISTIC REGRESSION

In [18]:
# fuente -> https://machinelearningmastery.com/cost-sensitive-logistic-regression/
model_LR = []

model = LogisticRegression(solver='lbfgs', class_weight='balanced')
for i in range(len(train_folds)):
    model.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"])
    joblib.dump(model, 'LR'+ str(i) +'.pkl')
    model_LR.append(model)
    predict = model.predict(test_folds[i].drop(columns=["hospital_death"]))

# Decision Tree Classifier

In [19]:
model_DT= []

for i in range(len(train_folds)):
    model = DecisionTreeClassifier(class_weight='balanced')
    model.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"])
    joblib.dump(model, 'DT'+ str(i) +'.pkl') 
    model_DT.append(model)
    predict = model.predict(test_folds[i].drop(columns=["hospital_death"]))

# knn

In [7]:
model_KKN = []
recall_KKN = []

for i in range(len(train_folds)):
    model = KNeighborsClassifier(n_neighbors=30)
    n_neighbors = range(1, 10, 3)
    weights = ['uniform', 'distance']
    metric = ['euclidean', 'manhattan', 'minkowski']
    #grid = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)
    #grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, scoring='recall',error_score=0)
    #grid_result = grid_search.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"])
    model.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"])
    model_KKN.append(model)
    joblib.dump(model, 'KNN'+ str(i) +'.pkl') 
    # summarize results
    """
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    """

Recall 0 fold: 0.996301
Recall 1 fold: 0.998329
Recall 2 fold: 0.996301
Recall 3 fold: 0.997494
Recall 4 fold: 0.995943
Recall 5 fold: 0.997375
Recall 6 fold: 0.999045
Recall 7 fold: 0.998210
Recall 8 fold: 0.998568
Recall 9 fold: 0.997016
mean Recall : 0.9974581811956872


# Stochastic Gradient Boosting

In [20]:
model_SGB = []
model = GradientBoostingClassifier()

for i in range(len(train_folds)):
    model.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"])
    joblib.dump(model, 'SGB'+ str(i) +'.pkl') 
    model_SGB.append(model)

# Cost-Sensitive Neural Network

In [48]:
# https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
# https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
# https://machinelearningmastery.com/cost-sensitive-neural-network-for-imbalanced-classification/

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def create_model(optimizer='adam', neurons=80):
    # define the keras model
    model = Sequential()
    model.add(Dense(180, input_dim=15, activation='relu'))
    model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile the keras model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=["accuracy"])
    return model

Using TensorFlow backend.


In [10]:
model_NR = []
recall_NR = []
model = KerasClassifier(build_fn=create_model, verbose=0)
batch_size = [2000]
neurons = [60,80]
epochs = [20]
param_grid = dict(batch_size=batch_size, epochs=epochs, neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
for i in range(len(train_folds)):
    grid_result = grid.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"], class_weight='balanced')
    # summarize results
    predict = grid_result.predict(test_folds[i].drop(columns=["hospital_death"]))
    recall = recall_score(test_folds[i]["hospital_death"], predict, average='binary')
    print('Recall %d fold: %f' % (i , recall))
    recall_NR.append(recall)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Recall 0 fold: 0.111111
Best: 0.915909 using {'batch_size': 2000, 'neurons': 80, 'epochs': 20}
0.915896 (0.008760) with: {'batch_size': 2000, 'neurons': 60, 'epochs': 20}
0.915909 (0.008865) with: {'batch_size': 2000, 'neurons': 80, 'epochs': 20}
Recall 1 fold: 0.070707
Best: 0.916030 using {'batch_size': 2000, 'neurons': 80, 'epochs': 20}
0.915884 (0.008820) with: {'batch_size': 2000, 'neurons': 60, 'epochs': 20}
0.916030 (0.008889) with: {'batch_size': 2000, 'neurons': 80, 'epochs': 20}
Recall 2 fold: 0.089646
Best: 0.915896 using {'batch_size': 2000, 'neurons': 80, 'epochs': 20}
0.915448 (0.008321) with: {'batch_size': 2000, 'neurons': 60, 'epochs': 20}
0.915896 (0.009117) with: {'batch_size': 2000, 'neurons': 80, 'epochs': 20}
Recall 3 fold: 0.080910
Best: 0.916006 using {'batch_size': 2000, 'neurons': 80, 'epochs': 20}
0.915885 (0.007049) with: {'batch_size': 2000, 'neurons': 60, 'e

In [21]:
model_SVM = []
for i in range(len(train_folds)):
    model = joblib.load('SVM' + str(i) + '.pkl')
    model_SVM.append(model)

# Stacking voting

In [ ]:
estimators = [
    ('nv', model_NC[0]),
    ('rf', model_RF[0]),
    ('svm',model_SVM[0]),
     ('lr',model_LR[0]),
     ('dt',model_DT[0]),
     ('knn',model_KKN[0]),
     ('sgb',model_SGB[0]),
]
model_SC = []

for i in range(len(train_folds)):
    model = clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
    model.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"])
    joblib.dump(model, 'stack'+ str(i) +'.pkl') 
    model_SC.append(model)
    print("Modelo stack: " + str(i) + "guardado")

Modelo stack: 0guardado
Modelo stack: 1guardado
Modelo stack: 2guardado
